In [1]:
import numpy as np
import pandas as pd
from collections import OrderedDict, defaultdict
import multiprocessing as mp
import glob
import sys, os
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
import gezi
from gezi import tqdm
tqdm.pandas()
import sentencepiece as spm
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [2]:
d = pd.read_csv('../input/info/infos.csv')

In [3]:
with open('../input/corpus.txt', 'w') as f:
  for row in tqdm(d.itertuples(), total=len(d)):
    print(row.title, file=f)
    print(row.asr_text, file=f)

  0%|          | 0/1138154 [00:00<?, ?it/s]

In [4]:
spm.SentencePieceTrainer.train(input='../input/corpus.txt', model_prefix='../input/sp10w', vocab_size=100000, user_defined_symbols=[f'SEP{i}' for i in range(200)])

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: ../input/corpus.txt
  input_format: 
  model_prefix: ../input/sp10w
  model_type: UNIGRAM
  vocab_size: 100000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  user_defined_symbols: SEP0
  user_defined_symbols: SEP1
  user_defined_symbols: SEP2
  user_defined_symbols: SEP3
  user_defined_symbols: SEP4
  user_defined_symbols: SEP5
  user_defined_symbols: SEP6
  user_defined_symbols: SEP7
  user_defined_symbols: SEP8
  user_defined_symbols: SEP9
  user_defined_symbols: SEP10
  user_defined_symbols: SEP11
  user_defined_symbols:

In [5]:
sp = spm.SentencePieceProcessor(model_file='../input/sp10w.model')
sp.encode('This is a test')

[22330, 63717, 5246, 205, 7545, 5246, 205, 7506, 205, 89475, 75999]

In [6]:
sp.decode([22130,101,102])

'日日SEP98SEP99'

In [7]:
for line in open('../input/corpus.txt'):
  line = line.strip()
  print(line)
  print(sp.encode(line))
  print(sp.encode(line, out_type=str))
  break

我只不过跳了跳舞，结果光子实在太狠了
[277, 6889, 7785, 670, 204, 408, 28086, 1394, 8143]
['▁我', '只不过', '跳了', '跳舞', ',', '结果', '光子', '实在', '太狠了']


In [8]:
vocab = gezi.Vocab('../input/sp10w.vocab', 0)

In [9]:
sp.encode('我只不过')

[277, 6889]

In [10]:
vocab.id('我')

211

In [11]:
vocab.id('光子')

28086

In [12]:
vocab.key(101)

'SEP98'

In [13]:
vocab_names = [
                'tag',
                'word',
              ]
vocabs = {}
for vocab_name in vocab_names:
  if vocab_name != 'word':
    vocab_file =  f'../input/{vocab_name}_vocab.txt'
    vocab = gezi.Vocab(vocab_file)
  else:
    vocab_file = '../input/sp10w.vocab'
    vocab = gezi.Vocab(vocab_file, 0)
  vocabs[vocab_name] = vocab

In [14]:
d.head()

,id,category_id,cat,subcat,tag_id,title,asr_text
0,8181862433864400302,10804,108,4,"55848798,800002360,593341,55744355,55859481,55...",我只不过跳了跳舞，结果光子实在太狠了,NaN
1,4155641279071878522,10401,104,1,"286226,101389,800000950,9504442,9820430",这么可爱的小姐姐，太搞怪了,邢飞真的是太能搞怪了，跟粉丝的速度一点都不装特别的，真就是喜欢这么正的女孩，你们是不是跟你一...
2,6281338907679991214,10501,105,1,"56080717,4093463,41614380,224444,101461,560840...",最近超级火的歌曲《冬眠》，美女唱得很不赖，绝对是实力派,你听。热车吗？你听着他说着你的。只上了赌注你。拍女生。啊。都要。的总结到位深。你替他掩饰着粽...
3,8506122561779211694,11604,116,4,"530228,3326413",下地干农活无意间拍下这一幕，这是什么东西，吓我一跳,一样。会落实。隆隆天空。给个反应。
4,4696077578276769146,10012,100,12,"101334,102376,800000171,10115859",和女儿们一块玩游戏，把她俩给骗了，她们却玩的很开心,你想。样子。认真在。让魔王魔王。是谁。


In [15]:
title_words = []
asr_words = []
for row in tqdm(d.itertuples(), total=len(d)):
  title_words.append(len(sp.encode(row.title)))
  asr_words.append(len(sp.encode(str(row.asr_text))))

  0%|          | 0/1138154 [00:00<?, ?it/s]

In [16]:
d['title_words'] = title_words
d['asr_words'] = asr_words

In [17]:
d.title_words.describe([.5,.9,.99,.999])

count    1.138154e+06
mean     9.539984e+00
std      4.032122e+00
min      1.000000e+00
50%      9.000000e+00
90%      1.500000e+01
99%      1.900000e+01
99.9%    2.400000e+01
max      4.200000e+01
Name: title_words, dtype: float64

In [18]:
d.asr_words.describe([.5,.9,.99,.999])

count    1.138154e+06
mean     2.367692e+01
std      2.940315e+01
min      1.000000e+00
50%      1.400000e+01
90%      5.700000e+01
99%      1.420000e+02
99.9%    2.090000e+02
max      1.984000e+03
Name: asr_words, dtype: float64

In [19]:
def gen_w2v(window=32, min_count=1, emb_dim=256):
  sentences = []
  for row in tqdm(d.itertuples(), total=len(d)):
    row = row._asdict()
    l = ['[CLS]', *sp.encode(str(row['title']), out_type=str), '[SEP]', *sp.encode(str(row['asr_text']), out_type=str), '[SEP]']
    sentences.append(l)
  ic(len(sentences))
  name = 'word'
  monitor = gezi.MonitorCallback(name) 
  w2v = Word2Vec(sentences, vector_size=emb_dim, window=window, min_count=min_count, sg=1, workers=mp.cpu_count(), epochs=10, callbacks=[monitor])
  ofile = f'../input/w2v/sp/{emb_dim}/{name}.pkl'
  gezi.try_mkdir(os.path.dirname(ofile))
  gezi.save_pickle(w2v, ofile)
  for name in vocabs:
    vocab = vocabs[name]
    # emb = np.zeros([vocab.size(), emb_dim])
    emb = np.random.uniform(-0.05, 0.05,(vocab.size(), emb_dim))
    for i in range(vocab.size()):
      word = vocab.key(i) 
      if word in w2v.wv:
        emb[i] = w2v.wv[word]
    ofile = f'../input/w2v/sp/{emb_dim}/{name}.npy'
    np.save(ofile, emb)
  
  return w2v

In [20]:
# gen_w2v(emb_dim=256)

In [21]:
# gen_w2v(emb_dim=512)

In [1]:
gen_w2v(emb_dim=300)

NameError: name 'gen_w2v' is not defined

In [ ]:
gen_w2v(emb_dim=400)

In [ ]:
gen_w2v(emb_dim=600)

In [22]:
gen_w2v(emb_dim=768)

  0%|          | 0/1138154 [00:00<?, ?it/s]

ic| 2606645293.py:7 in gen_w2v()- len(sentences): 1138154


name: word epoch: 1 model loss: 0.0 elapsed minutes: 24.27
name: word epoch: 2 model loss: 0.0 elapsed minutes: 28.84
name: word epoch: 3 model loss: 0.0 elapsed minutes: 28.59
name: word epoch: 4 model loss: 0.0 elapsed minutes: 28.83
name: word epoch: 5 model loss: 0.0 elapsed minutes: 26.66
name: word epoch: 6 model loss: 0.0 elapsed minutes: 25.17
name: word epoch: 7 model loss: 0.0 elapsed minutes: 24.44
name: word epoch: 8 model loss: 0.0 elapsed minutes: 24.06
name: word epoch: 9 model loss: 0.0 elapsed minutes: 23.99
